# SUMMARY OF FILE HANDLING IN Q

## 1. [Running q scripts](https://code.kx.com/q4m3/10_Execution_Control/#103-scripts)

- There are 3 ways to run q scripts:
    - from script with a q command: __\l /path/to/file.q__, this is load command:
        - does not work from kdb studio
        - only works with absolute path
    - from script with q's system command: system "l /path/to/file.q"
    - from terminal: ~$$: q /path/to/file.q arg1 arg2 ...
- The 1st and the 2nd is practically the equivalent of load module or import in other languages.

# 2. Reading and writing data files

- __Handling binary files__:
    - no instantiation of readers, writers or serializers
    - symbolic handels represents files: `: (backtick + colon + path + filename)` `:path/filename.ext
    - set: save / write function, which saves the data (list, table, etc) in a serialized binary data file (if success, echos symbolic file handle to stdout)
    - get: read function to read in a file (if successful, echos nothing) into q entities (list table etc)

- __[Handling txt files](http://178.62.21.29/q/ref/filenumbers/#0-text-files)__
    - saving strings to text files: "0:" operator
        - syntax: `fileHandle` 0: ("first line";"second line")
    - reading text files to strings: myVar:read0 `simbolic_file_handle
        - read0 will read the lines of a txt file as elements of a string list

- __[Handling csv files](http://178.62.21.29/q/ref/filenumbers/#load-csv)__
    - three different overloads of the 0: operator
        - writes list of strings to text files (see handling text files)
        - prepares the tables as text
            - q handles the quoting and escaping of special characters
        - parsing formatted csv into q table

In [ ]:
`:testfiles/tio.csv 0: csv 0: tio / prepares tables as text, creates a comma separeted text file, then writes it to a csv file

In [ ]:
csvString:read0 `:testfiles/tio.csv / reads csv file into list of strings

In [ ]:
tio_read:("SF"; enlist ",") 0: `:testfiles/tio.csv / reads csv file directly into q table with known column types (symbol and float)

- __3 ways to read a binary file into memory as a q entity__
    - \l path/to/file (NOT A FILE HANDLE!)
        - \l assigns the content of a file to an entity with the same name as the file
    - get filehandle
    - value `filehandle

- Writing and overwriting a file with _dot amend_ notation (same as set):
    - serializes q entities to a file
    - .: writes or overwrites a file
    - ,: appends to a file

# 11. File I/O

## 11.1. Bynary data

In [3]:
path:"/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/"
path_file:"/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/fox.us.csv"

In [1]:
 / cast: $
show `$"df df d f" / cast to symbol
hsym `$"df/fg rt/df.ext" / cast to symbolic file handle inserting a colon at the beginning of symbol: `$":fdf/df"
hsym `$"/data/file name.csv"
hsym hsym `$"/data/file name.csv" / idempotent: passes output through

`df df d f


`:df/fg rt/df.ext


`:/data/file name.csv


`:/data/file name.csv


In [4]:
hcount hsym `$path_file / hcount returns the size of file in bytes

158527


In [7]:
hdel hsym `$path,"t" / delete file by hanle name

`:/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/t


### 11.1.3. Serializing and seserializing

- every q entity can be serialized and persisted to storage
- write a q entity to binary file (create or overwrite file): set
    - filehandle set q entity
- read binary file as entity (three ways)
    - get filehandle
    - value filehandle
    - \l path/to/file (NOT A FILE HANDLE!)
        - \l assigns the content of a file to an entity with the same name as the file

### 11.1.4. Binary data files

- opening a symbolic handle returns a function, called an _open handle_, that is used to perform operations.
- hopen: returns the _open handle_ function
    - hopen filehandle (if file does not exist, creates it)
        - you can use the variable value with the open handle assigned to it to do the same operation as with the variable name
    - can open a websocket
- hclose: closes the open handle function
    - if handle refers to a websocket, it is only closed after all pending data is sent

- Append to file

In [8]:
file:hsym `$(path,"openHandle")
file set 10 20 30
h:hopen file
h[42]
h 100 200
hclose h / always close handle

`:/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/openHandle


5i


5i


In [9]:
get file

10 20 30 42 100 200


### Writing and reading binary

- read1 reads in binary data as a list of bytes. this show the internal representation of the serialized q entity
- 1: writes raw binary data to a binary file

In [10]:
read1 file

0xfe2007000000000003000000000000000a0000000000000014000000000000001e000000000..


In [ ]:
file2:hsym `$path,"answer.bin"
file2  1: 0x06072a
read1 file

- _dot amend_ (same as set):
    - serializes q entities to a file
    - .: writes or overwrites a file
    - ,: appends to a file

In [ ]:
.[file; (); :; 1001 1002 1003]
q)get file

In [ ]:
.[file; (); ,; 42]
get file

## 11.2. Save and load on tables
- to save a table into a binary file, use the same syntax as writing or reading any type of q entities (set, get)
- a simpler method is save and load: you do not have to specify the name of the returned table: it implicitely creates the variable / file
    - save filehandle (where the file name has to be the same as the table name)
    - load filehandle (where the resulting table name is the same as the filename)
    - if you specify an extension (__.txt, .csv, .xml or .xls__) it will be converted to that format
        - .txt table saved as tsv (tab separated values)
        - .csv table is saved as comma separated values
- you can save a table as csv:
    - filehandle 0: csv 0: table

In [ ]:
file set ([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
get file

In [14]:
system "touch fiot"

In [15]:
file3:hsym `$path,"fiot"

In [13]:
fiot:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)

In [16]:
save file3

`:/home/iguana/1_Code/4_jupyter_projects/q4m3_tutorial/data/fiot


In [17]:
load file3

`fiot


In [ ]:
 / save as .txt
file4:hsym `$path,"t.txt"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file4
read0 file4
("SIF"; enlist "\t") 0: file4  / no corresponding load or get. you have to parse

In [ ]:
 / save as .csv
file5:hsym `$path,"t.csv"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file5
read0 file5
("SIF"; enlist ",") 0: file5 / no corresponding load or get. you have to parse

In [ ]:
 / save as .xml (you have to use outside [libraries](http://code.kx.com/q/github) to read xml files)
file6:hsym `$path,"t.xml"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file6
read0 file6

In [ ]:
file7:hsym `$path,"t.xls"
t:([] c1:`a`b`c; c2:10 20 30; c3:1.1 2.2 3.3)
save file7
read0 file7

## 11.3. Splayed tables

- serialize large tables by their columns into separate files
- just use a trailing forward slash at the end of the path symbol set table
- Restrictions on tables that can be splayed:
    - All columns must be simple or compound lists (a list of simple lists of uniform type)
    - An arbitrary general list column cannot be splayed.
- Symbol columns must be enumerated.

In [ ]:
folder1:hsym `$path,"t_folder/"
t:([] c1:10 20 30; c2:1.1 2.2 3.3)
folder1 set t

## 11.4. Text data
### 11.4.3. Reading and writing text files

- read0 reads a .txt file as a list of strings
    - read0 `file.txt
- 0: writes a list of strings as a .txt file
    - `file.txt 0: tabel_entity

- use hopen and hclose to handle .txt files
- neg[h] (value1;value2) appends string values to the existing file

- saving to .csv file while controlling the filename (vs save hsym `table_name.csv):
    - `name.csv 0: csv 0: table

### 11.5. Parsing records

- parses text file with equal length rows into a table with columns of specified type and length
- Dyadic forms of 0: and 1: parse individual fields according to data type from text or binary records.
- Field parsing is based on the following [field types](https://code.kx.com/q4m3/11_IO/#115-parsing-records) !!!!

In [ ]:
data:("JFS D";4 7 10 7 10) 0: `:./data/fixed.txt / creates a nested list
flip `c1`c2`c3`c4!data / create table from the list of lists

### 14.5.2. Variable length records

### 14.5.3. Key-value records

In [18]:
ll1:"S=;" 0: "one=1;two=2;three=3"
"S:/" 0: "one:1/two:2/three:3"
"I=;" 0: "1=one;2=two;3=three"

one  two  three
,"1" ,"2" ,"3" 


1     2     3      
"one" "two" "three"


In [19]:
flip `k`v!ll1

k     v   
----------
one   ,"1"
two   ,"2"
three ,"3"
